In [77]:
import pandas as pd
import numpy as np
import ast
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [78]:
movies = pd.read_csv('dataset/tmdb_5000_movies.csv')

In [79]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10/12/2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19/5/2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26/10/2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16/7/2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",7/3/2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [80]:
def convert(obj):
    try:
        return [i['name'].lower() for i in ast.literal_eval(obj)]
    except:
        return []

In [81]:
movies['genres'] = movies['genres'].apply(convert)
movies['genres']

0       [action, adventure, fantasy, science fiction]
1                        [adventure, fantasy, action]
2                          [action, adventure, crime]
3                    [action, crime, drama, thriller]
4                [action, adventure, science fiction]
                            ...                      
4797                        [action, crime, thriller]
4798                                [comedy, romance]
4799               [comedy, drama, romance, tv movie]
4800                                               []
4801                                    [documentary]
Name: genres, Length: 4802, dtype: object

In [82]:
emotion_genre_map = {
    # Sorting emotions from positive to negative
    'Curious': ['mystery', 'documentary'],
    'Excited': ['adventure', 'fantasy', 'action', 'science fiction'],
    'Happy': ['adventure', 'animation'],
    'Hopeful': ['science fiction', 'biography'],
    'Inspirational': ['biography', 'war', 'history'],
    'Loving': ['romance', 'teen'],
    'Relieved': ['comedy', 'family'],
    'Surprised': ['mystery', 'fantasy', 'thriller', 'adventure'],
    
    'Angry': ['action', 'thriller'],
    'Bored': ['crime', 'tv movie', 'slice of life'],
    'Confused': ['mystery', 'psychological'],
    'Sad': ['drama', 'horror'],
    'Fearful': ['adventure', 'animation', 'comedy'],
    'Frustrated': ['crime', 'thriller'],
    'Lonely': ['foreign', 'drama', 'family'],
    'Nostalgic': ['history', 'war'],
    'Tense': ['thriller', 'mystery']
}

In [83]:
scaler = MinMaxScaler()
movies[['popularity', 'vote_average', 'vote_count']] = scaler.fit_transform(
    movies[['popularity', 'vote_average', 'vote_count']]
)

In [84]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[action, adventure, fantasy, science fiction]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",0.171815,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10/12/2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,0.72,0.858057
1,300000000,"[adventure, fantasy, action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",0.158846,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19/5/2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,0.69,0.327225
2,245000000,"[action, adventure, crime]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,0.122635,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26/10/2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,0.63,0.324753
3,250000000,"[action, crime, drama, thriller]",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,0.128272,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16/7/2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,0.76,0.662158
4,260000000,"[action, adventure, science fiction]",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",0.050169,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",7/3/2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,0.61,0.154450


In [85]:
# Define Features (X) and Target (y)
X = movies[['popularity', 'vote_average', 'vote_count']].values
y = movies['vote_average'] * movies['vote_count']  # Movie Score

In [86]:
# Split into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [87]:
# Reshape Input Data for LSTM (samples, time steps, features)
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [88]:
# Build LSTM Model
lstm_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train.shape[1])),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)  # Output: predicted movie score
])

c:\Users\Cyborg\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [89]:
lstm_model.compile(optimizer=Adam(learning_rate=0.001), 
                   loss=tf.keras.losses.MeanSquaredError(), 
                   metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [90]:
history = lstm_model.fit(
    X_train_reshaped, y_train,
    epochs=50, batch_size=32,
    validation_data=(X_test_reshaped, y_test),
    verbose=1
)

Epoch 1/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.0035 - mean_absolute_error: 0.0323 - val_loss: 4.4337e-04 - val_mean_absolute_error: 0.0146
Epoch 2/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3.0851e-04 - mean_absolute_error: 0.0117 - val_loss: 7.8289e-05 - val_mean_absolute_error: 0.0050
Epoch 3/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.4672e-04 - mean_absolute_error: 0.0072 - val_loss: 1.6651e-04 - val_mean_absolute_error: 0.0079
Epoch 4/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2.6353e-04 - mean_absolute_error: 0.0103 - val_loss: 5.5918e-05 - val_mean_absolute_error: 0.0049
Epoch 5/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.5402e-04 - mean_absolute_error: 0.0063 - val_loss: 1.0262e-04 - val_mean_absolute_error: 0.0062
Epoch 6/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.3776e-04 - mean_absolute_error: 0.0072 - val_loss: 5.6991e-05 - val_mean_absolute_error: 0.0037
Epoch 7/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - lo

In [91]:
y_pred_lstm = lstm_model.predict(X_test_reshaped)
mse_lstm = mean_squared_error(y_test, y_pred_lstm)
r2_lstm = r2_score(y_test, y_pred_lstm)

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


In [92]:
threshold = np.median(y_test)
y_test_class = (y_test > threshold).astype(int)
y_pred_class = (y_pred_lstm > threshold).astype(int)

In [93]:
accuracy = accuracy_score(y_test_class, y_pred_class)

In [94]:
# Save LSTM Model
lstm_model.save("lstm_movie_model.h5")

In [95]:
def get_recommendations(emotion, num_recommendations=100):
    if emotion not in emotion_genre_map:
        return ["Invalid emotion selected."]
    
    selected_genre = emotion_genre_map[emotion]
    filtered_movies = movies[
        movies['genres'].apply(lambda genres: any(g in genres for g in selected_genre))
    ].copy()

    if filtered_movies.empty:
        return ["No movies found for this emotion."]
    
    features = filtered_movies[['popularity', 'vote_average', 'vote_count']].values
    features_reshaped = features.reshape((features.shape[0], 1, features.shape[1]))
    
    scores = lstm_model.predict(features_reshaped)
    
    filtered_movies = filtered_movies.assign(score=scores.flatten())
    
    ranked_movies = filtered_movies.sort_values(by='score', ascending=False).head(num_recommendations)
    
    return ranked_movies[['id', 'title', 'genres', 'score']].values.tolist()

In [96]:
user_emotion = input("Emotion: ").strip()
recommendations = get_recommendations(user_emotion)

In [97]:
print(f"R² Score: {r2_lstm:.4f}")
print(f"Accuracy: {accuracy:.4f}")

R² Score: 0.9975
Accuracy: 0.9532
